[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jlfvindel/SW-KG/blob/main/RDFlib/Wikidata-CONSTRUCT-Colab.ipynb)

## Recepción de un grafo desde Wikidata

### Resumen de este cuaderno
+ Este cuaderno está diseñado para ser **ejecutado localmente**, no sobre Google Colab o servicios similares
+ Se remite una consulta a Wikidata de tipo CONSTRUCT, de la que se espera un grafo como respuesta
+ Se visualiza interactivamente el grafo con el paquete Jaal
+ Se conecta con una instalación local de GraphDB y se insertan allí los datos

### Contexto de trabajo

In [ ]:
# Comente o descomente conforme los paquetes estén instalados
!pip install sparqlwrapper rdflib 

## Diseño de la consulta

### Primero, como consulta SELECT
Aprovechando el interfaz web de consulta de Wikidata, se experimenta con una consulta SELECT previa, con las siguientes características:
+ se acotan las entidades de interés a los museos localizados en España
+ de estos ítem ?museo nos interesan todas las relaciones (museo, relacionado_por_X_con, objeto) almacenadas en Wikidata como enunciados directos; para todo tipo de relación posible del museo con todo tipo de objeto
+ adicionalmente se solicita el tipo de la relación, es decir, qué tipo sintáctico se espera como objeto, así como la clase semántica a la que pertenece el objeto

> **Esta consulta se puede ver y ejecutar externamente en** [este enlace](https://query.wikidata.org/#%23%201.%20Items%20%28%3Fmuseo%29%2C%20instancias%20de%20cualquier%20subclase%20de%20museo%20%28Q33506%29%20en%20pa%C3%ADs%20España%20%28Q29%29.%0A%23%20%20%20%20Se%20busca%20toda%20tripleta%20con%20%3Fmuseo%20como%20sujeto%3A%20%3Fmuseo%20%3FpD%20%3Fo%20.%0A%23%202.%20En%20realidad%20en%201%20se%20buscan%20tan%20sólo%20las%20tripletas%20que%20sean%20%27enunciados%20directos%27%3A%20%3Fs%20%3FpD%20%3Fo%2C%20as%C3%AD%20que%0A%23%20%20%20%20%3FpD%20debe%20cumplir%20una%20restricción.%20Si%20%3Fp%20wikibase%3AdirectClaim%20%3FpD%2C%20entonces%20%3FpD%20es%20la%20variante%20sintáctica%0A%23%20%20%20%20adecuada%20para%20ser%20usada%20en%20enunciados%20directos.%0A%23%203.%20Los%20objetos%20%3Fo%2C%20en%20caso%20de%20no%20ser%20literales%2C%20pueden%20pertenecer%20a%20alguna%20clase%0A%0ASELECT%20%20%3FmuseoLabel%20%3FpLabel%20%3FoLabel%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%3Fmuseo%20%3FpD%20%3Fo%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%3FpTipo%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%3FoClase%20%3FoClaseLabel%20%20%20%20%20%20%20%0AWHERE%20%7B%0A%20%20%3Fp%20rdf%3Atype%20wikibase%3AProperty%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23%202%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AdirectClaim%20%3FpD%20%3B%20%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3ApropertyType%20%3FpTipo.%20%0A%20%20%0A%20%20%3Fmuseo%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%23%201%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ29%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%3FpD%20%3Fo%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%0A%20%20OPTIONAL%20%7B%3Fo%20wdt%3AP31%20%3FoClase%7D%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23%203%0A%20%20%0A%20%20%23%20De%20cualquier%20%3Fitem%2C%20este%20servicio%20proporciona%20su%20etiqueta%20en%20%3FitemLabel%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22es%22%2C%22ca%22%2C%22eu%22%2C%22gl%22%2C%22en%22%2C%22fr%22.%20%7D%0A%7D%0AORDER%20BY%20%3Fmuseo%0ALIMIT%20200000). La consulta devuelve una colección de n-tuplas, con tantos componente como variables solicitadas como respuesta en SELECT.

### Diseño de la consulta CONSTRUCT
También esta consulta CONSTRUCT se puede ejecutar y refinar directamente sobre el interfaz web de Wikidata. El patrón WHERE de esta consulta es similar al anterior y devuelve una n-tupla resultante cada vez que una instanciación de todas las variables de ese patrón coincidan localmente con el grafo consultado (el de todo Wikidata).
> [Este enlace](https://query.wikidata.org/#CONSTRUCT%20%7B%0A%20%20%23%20Por%20el%20cuerpo%20del%20WHERE%2C%20los%20%3Fsujeto%20son%20museos%20en%20España.%0A%20%20%3Fsujeto%20%3FpropD%20%3Fobjeto%20%3B%0A%20%20%20%20%20%20%20%20%20%20rdf%3Atype%20wikibase%3AItem%20%3B%0A%20%20%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3FsujetoLabel%20%3B%0A%20%20%20%20%20%20%20%20%20%20rdf%3Atype%20%3FsujetoClase%20.%0A%20%20%3FsujetoClase%20rdf%3Atype%20rdfs%3AClass%20.%0A%20%20%20%0A%20%20%3Fprop%20rdf%3Atype%20wikibase%3AProperty%20%3B%0A%20%20%20%20%20%20%20%20rdf%3Atype%20rdf%3AProperty%20%3B%0A%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3FpropLabel%20%3B%0A%20%20%20%20%20%20%20%20wikibase%3AdirectClaim%20%3FpropD%20%3B%0A%20%20%20%20%20%20%20%20wikibase%3ApropertyType%20%3FpropEsperaObjeto%20.%0A%20%20%0A%20%20%3FobjetoItem%20rdfs%3Alabel%20%3FobjetoItemLabel%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP31%20%3FobjetoItemClase%20.%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%3FobjetoItemClase%20rdf%3Atype%20rdfs%3AClass%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3FobjetoItemClaseLabel%20.%20%20%0A%7D%0AWHERE%20%7B%0A%20%20%3Fprop%20rdf%3Atype%20wikibase%3AProperty%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AdirectClaim%20%3FpropD%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3ApropertyType%20%3FpropEsperaObjeto.%20%0A%20%20%0A%20%20%3Fsujeto%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP31%20%3FsujetoClase%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ29%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%3FpropD%20%3Fobjeto%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%0A%20%20OPTIONAL%20%7B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%3Fobjeto%20wdt%3AP31%20%3FobjetoItemClase%20.%0A%20%20%20%20BIND%28%3Fobjeto%20AS%20%3FobjetoItem%29%20%0A%20%20%7D%20.%20%20%20%20%20%20%20%20%20%20%0A%20%20%0A%20%20%23%20De%20cualquier%20%3Fitem%2C%20este%20servicio%20proporciona%20su%20etiqueta%20en%20%3FitemLabel%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22es%22%2C%22ca%22%2C%22eu%22%2C%22gl%22%2C%22en%22%2C%22fr%22.%20%7D%0A%7D) **muestra la consulta CONSTRUCT, que puede ejecutarse externamente**. 

En la consulta SELECT anterior, cada una de esas n-tuplas resultantes se mostraban como una fila de resultados de la tabla. Las consultas CONSTRUCT siempre devuelven tripletas (sujeto, propiedad, objeto), es decir, un grafo resultante. Las n-tuplas de instancias que va facilitando el cuerpo del WHERE se usan para configurar el grafo tal y como se desee en el cuerpo de CONSTRUCT: con los espacios de nombre que se desee y con la relación entre entidades que se quiera. En este ejemplo, salvo la inserción de alguna tripleta nueva, se mantiene en el grafo resultantela relación que tenían las entidades en Wikidata.

In [1]:
from SPARQLWrapper import SPARQLWrapper, RDFXML, TURTLE

In [2]:
consulta = '''
CONSTRUCT {
  ?sujeto ?propD ?objeto ;
          rdfs:label ?sujetoLabel ;
          wdt:P31 ?sujetoClase .
   
  ?prop rdf:type wikibase:Property ;
        rdfs:label ?propLabel ;
        wikibase:directClaim ?propD ;
        wikibase:propertyType ?propTipoObjeto .
  
  ?objetoItem rdfs:label ?objetoItemLabel ;
              wdt:P31 ?objetoItemClase .
  
  ?objetoClase rdfs:label ?objetoClaseLabel .  
}
# WHERE devuelve (?sujeto, ?sujetoClase, ?prop, ?propD, ?propTipoObjeto, ?objeto, ?objetoClase)
# y el SERVICE wikibase:label devuelve las etiquetas ?xxxLabel de estos ítem
WHERE {
  ?prop rdf:type wikibase:Property ;                 
            wikibase:directClaim ?propD ;             
            wikibase:propertyType ?propTipoObjeto. 
  
  ?sujeto wdt:P31/wdt:P279* wd:Q33506 ;              
          wdt:P31 ?sujetoClase ;                     
          wdt:P17 wd:Q29 ;                           
          ?propD ?objeto .                           
  
  # Si el objeto es de tipo Item, demandamos su clase 
  OPTIONAL {                                         
      ?objeto wdt:P31 ?objetoItemClase .
      BIND(?objeto AS ?objetoItem)
  }            
  # De cualquier ?item, este servicio proporciona su etiqueta en ?itemLabel 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "es","ca","eu","gl","en","fr". }
}
'''

In [3]:
servidor = "https://query.wikidata.org/sparql" 
cliente = SPARQLWrapper(servidor)
cliente.setReturnFormat(RDFXML)

cliente.setQuery(consulta)
res_grafo = cliente.query().convert()

In [5]:
res_turtle = res_grafo.serialize(format="turtle")

str

In [7]:
print(r[:20000])

@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix wikibase: <http://wikiba.se/ontology#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

wd:Q100268926 a rdfs:Class ;
    rdfs:label "poblado ibérico"@es .

wd:Q1007870 a rdfs:Class .

wd:Q10267336 a rdfs:Class ;
    rdfs:label "barrio administrativo de Madrid"@es .

wd:Q10358588 a rdfs:Class ;
    rdfs:label "regimiento de infantería"@es .

wd:Q1045481 a rdfs:Class .

wd:Q1047113 a rdfs:Class ;
    rdfs:label "especialidad"@es .

wd:Q105388829 a rdfs:Class ;
    rdfs:label "diócesis católica romana"@es .

wd:Q105390172 a rdfs:Class ;
    rdfs:label "archidiócesis metropolitana católica romana"@es .

wd:Q10541491 a rdfs:Class ;
    rdfs:label "forma legal"@es .

wd:Q1060829 a rdfs:Class ;
    rdfs:label "